# Movie Market Analysis

## Define Objectives

### Business Problem

Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

### Project Overview

The primary objective of this project is to explore and analyze trends in the movie industry to provide actionable insights for a new movie studio. This analysis aims to inform the studio's decision-making process regarding the types of films to produce, based on current market trends and box office performance.

### Key Business Questions

To guide the analysis, we will focus on the following key business questions:

1. **What types of films are currently performing well at the box office?**
   - Analyze the box office performance of different genres to identify which types of films generate the highest revenue.
   
2. **Which factors are associated with higher box office revenue?**
   - Investigate various factors such as budget, release date, and runtime to determine their impact on box office success.
   
3. **What actionable strategies can be derived from the analysis to guide the studio’s film production decisions?**
   - Provide concrete recommendations based on the data analysis to help the studio decide on the types of films to produce.

### Data Sources

To answer these questions, we will utilize datasets from the following sources:
- **Box Office Mojo**: Data on box office gross.
- **IMDB**: Movie metadata, including genres, ratings, and more (stored in a SQLite database).
- **Rotten Tomatoes**: Movie reviews and ratings.
- **TheMovieDB**: Additional movie details and metadata.
- **The Numbers**: Box office revenue data.

### Expected Deliverables

The expected deliverables for this project are:
1. **Non-Technical Presentation**: A slide deck presenting the analysis and findings to business stakeholders.
2. **Jupyter Notebook**: A well-documented notebook that presents the analysis in a clear and organized manner, suitable for a data science audience.
3. **GitHub Repository**: A repository containing all project files, including the notebook, presentation, and a detailed README file.

By addressing these objectives and utilizing the provided datasets, we aim to deliver insights that will guide the new movie studio in making informed production decisions.

## Data Acquisition

### Overview

In this section, we will acquire and load the datasets needed for our analysis. The data comes from multiple sources, including Box Office Mojo, IMDB, Rotten Tomatoes, TheMovieDB, and The Numbers. We will first download and unzip the datasets, then load them into our Jupyter Notebook for further analysis.

### Datasets

The primary datasets we will use are:
1. **Box Office Mojo**: Contains data on box office gross.
   - File: `data/bom.movie_gross.csv`
   
2. **IMDB**: Contains movie metadata including genres, ratings, and more.
   - File: `data/im.db`
   - Relevant tables: `movie_basics`, `movie_ratings`
   
3. **Rotten Tomatoes**: Contains movie reviews and additional metadata.
   - Files: 
     - `data/rt.movie_info.tsv` (Movie Info)
     - `data/rt.reviews.tsv` (Reviews)
   
4. **TheMovieDB**: Contains movie metadata including genres, release dates, and ratings.
   - File: `data/tmdb.movies.csv`
   
5. **The Numbers**: Contains data on production budgets and box office gross.
   - File: `data/tn.movie_budgets.csv`

### Data Loading

We will use Python libraries such as `pandas` and `sqlite3` to load the datasets into dataframes for analysis.

In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import os
import zipfile
import numpy as np
%matplotlib inline

In [11]:
bom_df = pd.read_csv('ZippedData/bom.movie_gross.csv.gz')
rtmi_df = pd.read_csv('ZippedData/rt.movie_info.tsv.gz', delimiter='\t', index_col=0)
rtr_df = pd.read_csv('ZippedData/rt.reviews.tsv.gz', delimiter='\t', index_col=0, encoding='ISO-8859-1')
tmdb_df = pd.read_csv('ZippedData/tmdb.movies.csv.gz', index_col=0)
tnmb_df = pd.read_csv('ZippedData/tn.movie_budgets.csv.gz')

In [18]:
# Unzip IMDB SQLite database
with zipfile.ZipFile('ZippedData/im.db.zip', 'r') as zip_ref:
    zip_ref.extractall('ZippedData')
print("IMDB database unzipped successfully.")

conn = sqlite3.connect('ZippedData/im.db')
pd.read_sql(
"""
SELECT name 
FROM sqlite_master 
WHERE type='table';
""", conn
)

IMDB database unzipped successfully.


,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


### Data Overview

#### Box Office Mojo Data

In [19]:
bom_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


#### IMDB Data

In [28]:
imdb_df = pd.read_sql(
"""
SELECT * 
FROM movie_basics
JOIN movie_ratings
USING (movie_id)
""", conn
)
imdb_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119


#### Rotten Tomatoes

##### Rotten Tomatoes: Movie_Info

In [22]:
rtmi_df.head()

,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
id,,,,,,,,,,,
1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


##### Rotten Tomatoes: Reviews

In [23]:
rtr_df.head()

,review,rating,fresh,critic,top_critic,publisher,date
id,,,,,,,
3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


#### TheMovieDB

In [24]:
tmdb_df.head()

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


#### The Numbers

In [25]:
tnmb_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


## Q1: Which genres are most profitable?

### Introduction

In this section, we will analyze the profitability of different movie genres. This involves examining the box office gross (both domestic and foreign), production budgets, and other relevant financial metrics. Our goal is to determine which genres consistently yield the highest returns on investment.

**Columns of Interest**
- genres (IMDB)
- domestic_gross (Box Office Mojo, The Numbers)
- foreign_gross (Box Office Mojo)
- worldwide_gross (The Numbers)

In [38]:
def get_percentage_missing(dataframe):
    missing_values_series = dataframe.isna().sum()
    missing_values_indexes = list(missing_values_series.index)
    missing_values_values = list(missing_values_series.values)
    missing_values_percentage = pd.DataFrame({"indexes": missing_values_indexes, "values": missing_values_values, "percentage_missing": list(np.round(((np.array(missing_values_values) * 100) / len(dataframe)), 2))})
    missing_values_percentage.set_index("indexes", inplace=True)
    return missing_values_percentage

In [39]:
get_percentage_missing(bom_df)

,values,percentage_missing
indexes,,
title,0,0.00
studio,5,0.15
domestic_gross,28,0.83
foreign_gross,1350,39.86
year,0,0.00


**Fix currency and type**

In [58]:
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [72]:
def fix_dollar(money):
    if isinstance(money, str):
        return float(money.replace('$', '').replace(',', ''))
    return money

In [73]:
tnmb_df['domestic_gross'] = tnmb_df['domestic_gross'].map(fix_dollar)
tnmb_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000",760507625.0,"$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000",241063875.0,"$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000",42762350.0,"$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000",459005868.0,"$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000",620181382.0,"$1,316,721,747"


In [80]:
merged_df = pd.merge(bom_df, tnmb_df, how='inner', left_on='title', right_on='movie')
# merged_df = pd.merge(merged_df, imdb_df, how='inner', left_on='title', right_on='primary_title')
merged_df
# merged_df = merged_df[['title', 'genres', 'domestic_gross', 'foreign_gross', 'production_budget', 'worldwide_gross']]
# merged_df.columns

,title,studio,domestic_gross_x,foreign_gross,year,id,release_date,movie,production_budget,domestic_gross_y,worldwide_gross
0,Toy Story 3,BV,415000000.0,652000000,2010,47,"Jun 18, 2010",Toy Story 3,"$200,000,000",415004880.0,"$1,068,879,522"
1,Inception,WB,292600000.0,535700000,2010,38,"Jul 16, 2010",Inception,"$160,000,000",292576195.0,"$835,524,642"
2,Shrek Forever After,P/DW,238700000.0,513900000,2010,27,"May 21, 2010",Shrek Forever After,"$165,000,000",238736787.0,"$756,244,673"
3,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010,53,"Jun 30, 2010",The Twilight Saga: Eclipse,"$68,000,000",300531751.0,"$706,102,828"
4,Iron Man 2,Par.,312400000.0,311500000,2010,15,"May 7, 2010",Iron Man 2,"$170,000,000",312433331.0,"$621,156,389"
...,...,...,...,...,...,...,...,...,...,...,...
1242,Gotti,VE,4300000.0,NaN,2018,64,"Jun 15, 2018",Gotti,"$10,000,000",4286367.0,"$6,089,100"
1243,Ben is Back,RAtt.,3700000.0,NaN,2018,95,"Dec 7, 2018",Ben is Back,"$13,000,000",3703182.0,"$9,633,111"
1244,Bilal: A New Breed of Hero,VE,491000.0,1700000,2018,100,"Feb 2, 2018",Bilal: A New Breed of Hero,"$30,000,000",490973.0,"$648,599"
1245,Mandy,RLJ,1200000.0,NaN,2018,71,"Sep 14, 2018",Mandy,"$6,000,000",1214525.0,"$1,427,656"


#### Fix issue with domestic_gross

In [77]:
tnmb_df.loc[(tnmb_df['movie']) == 'Avatar']

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000",760507625.0,"$2,776,345,279"


In [79]:
bom_df.loc[(bom_df['title']) == 'Avatar']

,title,studio,domestic_gross,foreign_gross,year
